CSC480 Assignment 1 tutorial
Damien Trunkey, Emily Lucas, Sophia Parrett, Fernando Valdivia, Divya Satrawada, Rey Ortiz
Social Justice League!

In [78]:
# Imports
import pandas as pd
import numpy as np
from collections import Counter
import sys
import time

In [79]:
# Get the data into a dataframe to manupulate and do preprocessing
dataFile = 'AmItheAsshole_subreddit.csv'
df = pd.read_csv(dataFile)
df = df.head(7)
df = df[['selftext', 'ups']]
df = df[2:7]
display(df)

,selftext,ups
2,Hello.\r\n\r\nI am one of the many in the midw...,14569
3,I (31f) married my husband (35m) when I gradua...,3022
4,\r\nI am a college student and in high school ...,3507
5,\r\n\r\nI (29F) got set up (again) for a blind...,1835
6,My wife (39F) and I (36M) got married and move...,11710


Explain normalization

In [80]:
bag_of_words = (
    df['selftext'].
    str.lower().                  # convert all letters to lowercase
    str.replace("[^\w\s]", " ").  # replace non-alphanumeric characters by whitespace
    str.split()                   # split on whitespace
)

<ipython-input-80-7300fddb91f6>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['selftext'].


Explain raw term frequency

In [84]:
raw_frequency = bag_of_words.apply(Counter)

df['selftext'] = raw_frequency

tf = pd.DataFrame(list(raw_frequency),index=raw_frequency.index)
tf = tf.fillna(0)
display(tf)
display(df)

,hello,i,am,one,of,the,many,in,midwest,hit,...,respects,admires,okay,feels,berate,allowing,happen,rebuild,boundaries,ashamed
2,1.0,40,2.0,2.0,13,14,1.0,5,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,27,2.0,0.0,6,12,0.0,13,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,65,1.0,1.0,7,14,2.0,16,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,20,0.0,1.0,3,8,0.0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,21,0.0,0.0,10,12,0.0,7,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


,selftext,ups
2,"{'hello': 1, 'i': 40, 'am': 2, 'one': 2, 'of':...",14569
3,"{'i': 27, '31f': 1, 'married': 1, 'my': 22, 'h...",3022
4,"{'i': 65, 'am': 1, 'a': 17, 'college': 6, 'stu...",3507
5,"{'i': 20, '29f': 1, 'got': 5, 'set': 1, 'up': ...",1835
6,"{'my': 8, 'wife': 4, '39f': 1, 'and': 17, 'i':...",11710


Explain Stopword and stemming removal

In [85]:
stopFile = 'stopwords-short.txt'
f = open(stopFile, "r")
stop_words = []
for line in f:
    words = line.split(',')
for word in words:
    word = word.replace('"', "").strip(" ")
    stop_words.append(word)

Explain the tf-idf vectors and why we use instead of raw frequencies